In [1]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser

In [3]:
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [4]:
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  
    max_tokens=200,              
)

In [5]:
pdf_path = "os-sertoes.pdf"
loader = PyPDFLoader(pdf_path, extract_images=False)
pages = loader.load_and_split()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,          
    chunk_overlap=20,        
    length_function=len,      
    add_start_index=True,     
)

chunks = text_splitter.split_documents(pages)

db = Chroma.from_documents(chunks, embedding=embeddings_model)


In [7]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [8]:
TEMPLATE = """
Responda a pergunta

Contexto: {context}
    
Pergunta: {question}
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=TEMPLATE)
parser = StrOutputParser()
sequence = RunnableSequence(prompt | llm | parser)

In [9]:
def answer_question(question: str):
    context = retriever.invoke(question)
    response = sequence.invoke({"context": context, "question": question})
    return response

In [10]:
for index, question in enumerate(questions):
    resposta = answer_question(question)
    print({"numero": index, "pergunta": question, "resposta": resposta})

{'numero': 0, 'pergunta': 'Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?', 'resposta': 'Euclides da Cunha tinha uma visão bastante crítica sobre o ambiente natural do sertão nordestino, descrevendo-o como um local marcado por secas e dificuldades extremas. Ele via o sertão como um ambiente que impõe condições adversas aos habitantes locais, resultando em sofrimento e tragédias, como no caso da campanha de Canudos. O ambiente árido e inóspito do sertão nordestino influencia diretamente a vida dos habitantes, tornando a sobrevivência uma verdadeira luta diária, marcada pela falta de recursos naturais e pelas condições climáticas severas.'}
{'numero': 1, 'pergunta': 'Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?', 'resposta': 'Euclides da Cunha descreve a população sertaneja como sendo resil